In [2]:
#imports

import numpy as np 
import pandas as pd
import glob
import plotly.express as px
import requests


%load_ext autoreload
%autoreload 2

In [18]:
df = pd.read_csv("../raw/caiso_merged_hourly_2019_2025_with_weather_and_solar.csv")
df.head()

,time,Palm Springs,Redding,Sacramento,San Diego,Los Angeles,Fresno,San Francisco,Riverside,San Francisco to Sacramento,...,Month,Holiday,San Diego Solar Radiation,San Francisco Solar Radiation,Palm Springs Solar Radiation,Los Angeles Solar Radiation,Riverside Solar Radiation,Sacramento Solar Radiation,Redding Solar Radiation,Fresno Solar Radiation
0,2018-12-31 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2018-12-31 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2018-12-31 02:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2018-12-31 03:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2018-12-31 04:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
df = df.dropna()


In [20]:
df.head()

,time,Palm Springs,Redding,Sacramento,San Diego,Los Angeles,Fresno,San Francisco,Riverside,San Francisco to Sacramento,...,Month,Holiday,San Diego Solar Radiation,San Francisco Solar Radiation,Palm Springs Solar Radiation,Los Angeles Solar Radiation,Riverside Solar Radiation,Sacramento Solar Radiation,Redding Solar Radiation,Fresno Solar Radiation
24,2019-01-01 00:00:00,3.0,4.7,5.5,10.5,6.3,1.7,8.2,7.5,-2.7,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25,2019-01-01 01:00:00,3.3,4.5,5.3,10.2,5.5,3.3,7.7,7.8,-2.4,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26,2019-01-01 02:00:00,4.3,4.2,4.8,9.2,4.5,2.0,7.2,7.5,-2.4,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
27,2019-01-01 03:00:00,2.8,4.1,4.3,9.3,3.3,2.0,6.9,7.1,-2.6,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28,2019-01-01 04:00:00,3.5,3.8,4.1,8.7,2.7,-0.5,6.8,6.6,-2.7,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
#normalize the data by subtracting the mean and dividing by the std
def normalize(df):
    # normalize the columns except 'Month', 'time', 'Holiday', and 'Day of Week'
    exclude_cols = ['Month', 'time', 'Holiday', 'Day of Week']
    numerical_cols = [col for col in df.select_dtypes(include=[np.number]).columns if col not in exclude_cols]
    for col in numerical_cols:
        df[col] = (df[col] - df[col].mean()) / df[col].std()
    return df

df = normalize(df)
df.head()



,time,Palm Springs,Redding,Sacramento,San Diego,Los Angeles,Fresno,San Francisco,Riverside,San Francisco to Sacramento,...,Month,Holiday,San Diego Solar Radiation,San Francisco Solar Radiation,Palm Springs Solar Radiation,Los Angeles Solar Radiation,Riverside Solar Radiation,Sacramento Solar Radiation,Redding Solar Radiation,Fresno Solar Radiation
24,2019-01-01 00:00:00,-2.202404,-1.263321,-1.259060,-1.317289,-1.712844,-1.694238,-1.446927,-1.346714,-0.960101,...,1.0,1.0,-0.73265,-0.723064,-0.75795,-0.730498,-0.735514,-0.718219,-0.70097,-0.73269
25,2019-01-01 01:00:00,-2.170377,-1.283153,-1.281830,-1.381739,-1.837574,-1.534303,-1.586762,-1.308643,-0.911159,...,1.0,1.0,-0.73265,-0.723064,-0.75795,-0.730498,-0.735514,-0.718219,-0.70097,-0.73269
26,2019-01-01 02:00:00,-2.063621,-1.312902,-1.338756,-1.596570,-1.993488,-1.664250,-1.726597,-1.346714,-0.911159,...,1.0,1.0,-0.73265,-0.723064,-0.75795,-0.730498,-0.735514,-0.718219,-0.70097,-0.73269
27,2019-01-01 03:00:00,-2.223755,-1.322818,-1.395681,-1.575087,-2.180583,-1.664250,-1.810498,-1.397474,-0.943787,...,1.0,1.0,-0.73265,-0.723064,-0.75795,-0.730498,-0.735514,-0.718219,-0.70097,-0.73269
28,2019-01-01 04:00:00,-2.149026,-1.352567,-1.418451,-1.703985,-2.274131,-1.914149,-1.838464,-1.460925,-0.960101,...,1.0,1.0,-0.73265,-0.723064,-0.75795,-0.730498,-0.735514,-0.718219,-0.70097,-0.73269


In [15]:
#now plot the solar radiation and temperature for the first 1000 rows
fig = px.line(df[:1000], x='time', y=['Sacramento', 'Sacramento Solar Radiation'],)
fig.update_layout(title='Normalized Solar Radiation and Temperature in Sacramento (First 1000 Rows)',
                  xaxis_title='Time',
                  yaxis_title='Normalized Value',
                  legend_title='Variables')
fig.show()

So it looks like solar radiation and temerature are in sync with eachother so the time wasn't messed up by the time change.  The solar radiation is a little higher than the temperature, but that is expected since the solar radiation is measured in W/m^2 and the temperature is in degrees C.

In [22]:
#now let's try and predict load from this data:

#convert day of week, month, and holiday to categorical variables
df['Day of Week'] = df['Day of Week'].astype('category')
df['Month'] = df['Month'].astype('category')
df['Holiday'] = df['Holiday'].astype('category')



In [23]:
#now let's save this as the processed data
df.to_csv("../processed/caiso_merged_hourly_2019_2025_with_weather_and_solar_normalized.csv", index=False)
#and also save the columns that were used for normalization


In [ ]:
#clean the load data a bit: 
load_df = pd.read_csv("../raw/caiso_load_2019_2025.csv")